导入所需最少依赖。

# Search Agent

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re
import requests  # 新增用于真实搜索

LOCAL_MODEL_PATH = r"/root/autodl-tmp/Qwen3-8B"

定义并加载最小模型+分词器函数。

In [2]:
def load():
    tok = AutoTokenizer.from_pretrained(LOCAL_MODEL_PATH, local_files_only=True)
    m = AutoModelForCausalLM.from_pretrained(LOCAL_MODEL_PATH, local_files_only=True, device_map="auto" if torch.cuda.is_available() else None)
    return m, tok

model, tokenizer = load()

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


定义最小生成函数，返回原始与处理后输出。

In [3]:
def generate(prompt, max_new_tokens=512):
    msgs = [{"role":"user","content":prompt}]
    if hasattr(tokenizer, "apply_chat_template"):
        text = tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
    else:
        text = prompt
    inp = tokenizer(text, return_tensors="pt")
    for k,v in inp.items(): inp[k] = v.to(model.device)
    ids = model.generate(**inp, max_new_tokens=max_new_tokens)
    gen_ids = ids[0][inp['input_ids'].shape[-1]:]  # 仅保留新生成部分
    raw_text = tokenizer.decode(gen_ids, skip_special_tokens=False)
    clean_text = tokenizer.decode(gen_ids, skip_special_tokens=True)
    return gen_ids.tolist(), raw_text, clean_text


def clear_model_cache():
    """清空模型的KV缓存,让模型忘记之前的上下文"""
    if hasattr(model, 'past_key_values'):
        model.past_key_values = None
    # 清空CUDA缓存(如果使用GPU)
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

执行一次简单测试并输出原始与处理后结果，确保模型导入和基本功能正常。

In [4]:
tokens, raw_text, clean_text = generate("今天我心情很好")
print("RAW_ALL_TOKENS_TEXT:\n", raw_text)
print("\n\nCLEAN_RESPONSE:\n", clean_text)

RAW_ALL_TOKENS_TEXT:
 <think>
好的，用户今天心情很好，我需要回应他的好心情。首先，应该表达开心，让他感受到被理解。可以提到一些愉快的事情，比如阳光、微风，或者一起分享快乐。然后，可以询问他今天有什么开心的事情，这样能促进交流，让他更愿意分享。同时，保持语气轻松友好，用表情符号增加亲切感。还要注意不要过于复杂，保持自然，避免使用生硬的语言。最后，确保回应积极，鼓励他继续分享，或者一起寻找更多快乐的事情。这样用户会觉得被关心，也会更愿意继续对话。
</think>

哇，今天心情这么好呀！是不是有什么开心的事情发生啦？☀️微风轻轻吹，阳光暖暖照，感觉连空气都在笑呢！要不要跟我分享一下今天的小确幸呀？说不定还能一起找到更多让心情变好的小秘密呢～ 😄<|im_end|>


CLEAN_RESPONSE:
 <think>
好的，用户今天心情很好，我需要回应他的好心情。首先，应该表达开心，让他感受到被理解。可以提到一些愉快的事情，比如阳光、微风，或者一起分享快乐。然后，可以询问他今天有什么开心的事情，这样能促进交流，让他更愿意分享。同时，保持语气轻松友好，用表情符号增加亲切感。还要注意不要过于复杂，保持自然，避免使用生硬的语言。最后，确保回应积极，鼓励他继续分享，或者一起寻找更多快乐的事情。这样用户会觉得被关心，也会更愿意继续对话。
</think>

哇，今天心情这么好呀！是不是有什么开心的事情发生啦？☀️微风轻轻吹，阳光暖暖照，感觉连空气都在笑呢！要不要跟我分享一下今天的小确幸呀？说不定还能一起找到更多让心情变好的小秘密呢～ 😄


In [4]:
# Search Agent v2.0 core (真实搜索集成 + 多轮引用追踪)

# 长度控制参数
MAX_SOURCES_PER_SEARCH = 5  # 每次搜索返回的最大信息条数
MAX_SOURCE_DESC_LEN = 400    # 单条信息description的最大长度
MAX_FORMATTED_SOURCES_LEN = 1500  # 格式化搜索结果在prompt中的最大长度
MAX_RAW_DISPLAY_LEN = 1000   # 显示raw_responses的最大长度

BASE_PROMPT = (
    "你是一个智能搜索助手。你的任务是分析用户问题,判断是否需要搜索来获取信息。\n\n"
    "**核心判断原则:**\n"
    "在分析问题时,如果遇到以下情况,必须使用<search>标签搜索:\n"
    "1. 需要实时数据(股价、天气、新闻、汇率等会变化的信息)\n"
    "2. 需要最新信息(当前状态、今天/现在的情况)\n"
    "3. 需要具体事实(某个公司的数据、某个地点的情况、某个产品的参数)\n"
    "4. 你不确定答案,或答案可能过时\n"
    "5. 用户明确要求查询、搜索、查找信息\n\n"
    "如果是以下情况,可以直接回答:\n"
    "1. 常识性问题(如何做某事、概念解释)\n"
    "2. 主观问题(建议、意见)\n"
    "3. 数学计算、逻辑推理\n\n"
    "**确定需要搜索时的输出格式**:\n"
    "<search>简短查询词</search>\n\n"
    "**不需要搜索时,直接给出答案。**\n\n"
    "**关键规则:**\n"
    "- <search>标签单独成行,内容3-10字\n"
    "- 没搜索前不要编造数字、日期等事实\n"
    "- 宁可多搜索,不要猜测\n"
    "- 如果已有足够信息,直接回答并列出参考来源,不要再输出<search>\n"
)

# Jina Search API 配置
JINA_API_KEY = "jina_800f62ec9cc745e09f058c4652a961feziG6FeCa71toa9my7gXm3prQbJaF"
JINA_SEARCH_ENDPOINT = "https://s.jina.ai"
PROXIES = {"http": "http://127.0.0.1:7890", "https": "http://127.0.0.1:7890"}


def fetch_search_result(query: str, use_proxy: bool = True, max_sources: int = MAX_SOURCES_PER_SEARCH) -> dict:
    """调用 Jina Search 获取结果并返回结构化字典。失败时返回错误信息。"""
    headers = {
        "Accept": "application/json",
        "Authorization": f"Bearer {JINA_API_KEY}",
        "X-Respond-With": "no-content"
    }
    url = f"{JINA_SEARCH_ENDPOINT}/?q={requests.utils.quote(query)}"
    proxies_cfg = PROXIES if use_proxy else None
    try:
        resp = requests.get(url, headers=headers, proxies=proxies_cfg, timeout=12)
        resp.raise_for_status()
        results_json = resp.json()
        data = results_json.get('data', [])
        if not data:
            return {"error": f"未找到结果: {query}", "sources": {}}
        sources = {}
        for i, item in enumerate(data[:max_sources], 1):
            title = (item.get('title') or '').strip()[:120]
            description = (item.get('description') or '').strip()[:MAX_SOURCE_DESC_LEN]
            url_link = (item.get('url') or '').strip()
            if description and url_link:
                sources[f"信息{i}"] = {"url": url_link, "description": description, "title": title}
        return {"sources": sources, "error": None} if sources else {"error": "结果无有效内容", "sources": {}}
    except Exception as e:
        return {"error": f"{type(e).__name__}: {e}", "sources": {}}


def extract_search_query(raw: str):
    """提取<search>标签,支持单独成行或内嵌格式"""
    match = re.search(r'<search>\s*([^<\n]+?)\s*</search>', raw, re.IGNORECASE)
    if match:
        query = match.group(1).strip()
        if query and len(query) <= 80:
            return query, raw[:match.end()]
    return None, None


def clean_final_response(text: str) -> str:
    """清理最终回答,移除 <think> 和 <search> 标签及其内容,只保留用户可见部分。"""
    text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r'<search>.*?</search>', '', text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r'</?(?:think|search)>', '', text, re.IGNORECASE)
    text = '\n'.join(line.strip() for line in text.split('\n') if line.strip())
    return text.strip()


def format_sources_for_prompt(sources_dict: dict, used_sources: dict) -> str:
    """格式化信息字典为 prompt 字符串,并追踪已使用的来源。"""
    if not sources_dict:
        return "[无可用信息]"
    lines = ["**搜索结果:**"]
    for key, val in sources_dict.items():
        desc = val.get('description', '')[:300]
        url = val.get('url', '')
        title = val.get('title', '未命名')
        lines.append(f"{key}({title}): {desc}")
        lines.append(f"  URL: {url}")
        if url and url not in used_sources:
            used_sources[url] = title
    return "\n".join(lines)


def run_search_agent(user_query, max_rounds=5, max_new_tokens=512, use_proxy=True):
    # 清空模型缓存,确保每个新查询都是独立的上下文
    clear_model_cache()
    
    rounds = []
    prompt = f"{BASE_PROMPT}\n用户问题: {user_query}"
    search_count = 0
    used_sources = {}
    
    for r in range(1, max_rounds + 1):
        _, raw, clean = generate(prompt, max_new_tokens=max_new_tokens)
        entry = {"round": r, "raw": raw, "clean": clean, "prompt": prompt}  # 保存完整prompt
        
        q, raw_trunc = extract_search_query(raw)
        if q:
            search_count += 1
            entry['raw'] = raw_trunc if raw_trunc else raw
            entry['search'] = q
            
            result_dict = fetch_search_result(q, use_proxy=use_proxy)
            entry['search_result'] = result_dict
            
            if result_dict.get('error'):
                print(f"⚠️ 搜索失败: {result_dict['error']}")
                rounds.append(entry)
                break
            
            sources = result_dict.get('sources', {})
            formatted_sources = format_sources_for_prompt(sources, used_sources)
            entry['formatted_sources'] = formatted_sources[:MAX_FORMATTED_SOURCES_LEN]
            rounds.append(entry)
            
            # 构造下一轮 prompt - 完整详细地说明任务和当前状态
            prompt = (
                f"{BASE_PROMPT}\n\n"
                f"=== 任务回顾 ===\n"
                f"用户的原始问题(query)是: {user_query}\n\n"
                f"你是一个智能搜索助手,负责为用户查询信息并给出专业答案。\n"
                f"你在第{search_count}轮搜索中已经查询了关键词'{q}',获得了以下搜索结果:\n\n"
                f"{formatted_sources[:MAX_FORMATTED_SOURCES_LEN]}\n\n"
                f"=== 当前任务(极其重要!) ===\n"
                f"现在你需要分析这些搜索结果,判断是否足够回答用户的问题:\n\n"
                f"**情况1: 信息已经足够 (这是最核心的环节!)**\n"
                f"如果以上搜索结果包含了足够的信息来回答'{user_query}',请务必按以下步骤操作:\n\n"
                f"第一步(最重要!必须执行!): 直接回答用户问题(query)\n"
                f"  - 用1-3句话给出核心答案,包含具体数字/事实，回答要围绕query，简明扼要\n"
                f"  - 例如用户问股价,你要说'特斯拉(TSLA)当前股价为XXX美元'\n"
                f"  - 严禁照抄输出示例！一定要根据先前的所有信息和分析自己总结！\n"
                f"  - 不要跳过这一步!这是用户最需要的!\n\n"
                f"第二步: 补充详细说明(可选)\n"
                f"  - 如果有额外有价值的信息,简要补充\n"
                f"  - 如涨跌幅、市值等相关数据\n\n"
                f"第三步: 列出参考来源\n"
                f"  - 格式:\n"
                f"    参考来源:\n"
                f"    [1] URL1\n"
                f"    [2] URL2\n\n"
                f"第四步: 不要再输出<search>标签\n\n"
                f"输出示例:\n"
                f"特斯拉(TSLA)当前股价为XXX美元,较前一交易日上涨XX%。\n\n"
                f"参考来源:\n"
                f"[1] https://...\n"
                f"[2] https://...\n\n"
                f"**情况2: 信息不足,需要继续搜索**\n"
                f"如果搜索结果不够详细,或者缺少关键信息:\n"
                f"1. 分析缺少什么信息\n"
                f"2. 输出<search>新的查询词</search>来获取更多细节\n"
                f"3. 注意不要重复搜索相同的关键词\n\n"
                f"=== 关键提醒 ===\n"
                f"- 这些搜索结果都是你自己查询得到的,不是用户提供的\n"
                f"- 你必须给出实质性答案,不能只列出链接!用户需要你的总结!\n"
                f"- 信息足够就立即回答,先答案后链接,不要只有链接没有答案!\n"
                f"- 回答要专业简洁,直接针对问题,不要说'我需要分析'这类话\n\n"
                f"现在请立即给出你的答案(记住:先回答问题,再列参考来源):")
            continue
        
        # 无 <search> 标签,认为是最终答案
        entry['clean'] = clean_final_response(clean)
        entry['used_sources'] = used_sources.copy()
        rounds.append(entry)
        break
    
    return rounds, used_sources


def show_rounds(rounds, used_sources=None, user_query=None):
    if user_query:
        print(f"Query: {user_query}")
    
    for i, info in enumerate(rounds, 1):
        print(f"\n{'='*80}")
        print(f"=== Round {info['round']} ===")
        
        print('Raw_responses:')
        print(info['raw'][:MAX_RAW_DISPLAY_LEN] + '...' if len(info['raw']) > MAX_RAW_DISPLAY_LEN else info['raw'])
        
        if 'search' in info:
            print(f"\n🔍 search_content: {info['search']}")
        
        if 'search_result' in info:
            result = info['search_result']
            if result.get('error'):
                print(f"\n⚠️ 搜索错误: {result['error']}")
            else:
                sources = result.get('sources', {})
                print(f"\n📚 搜索到 {len(sources)} 条信息（仅展示前3条）:")
                for key, val in list(sources.items())[:3]:
                    print(f"  {key}: {val.get('description', '')[:100]}...")
        
        print()
        is_last = (i == len(rounds))
        if is_last:
            print('✅ Clean_responses (最终答案):')
            print(info['clean'] if info['clean'] else '[无最终回答]')
            if used_sources:
                print(f"\n📎 本次对话所有追踪的链接 (共{len(used_sources)}个):")
                for idx, (url, title) in enumerate(list(used_sources.items())[:], 1):
                    print(f"  [{idx}] {title}\n      {url}")
        print('='*80)


In [6]:
# Tests for Search Agent v2.0
print("\n### Test 1: 心情真好 ###")
user_query = "今天我心情真好"
rounds1, sources1 = run_search_agent(user_query)
show_rounds(rounds1, sources1, user_query)



### Test 1: 心情真好 ###
Query: 今天我心情真好

=== Round 1 ===
Raw_responses:
<think>
好的，用户说“今天我心情真好”，我需要分析是否需要搜索。首先，根据核心判断原则，用户的问题是关于心情的主观感受，属于主观问题，不需要实时数据或具体事实。用户并没有要求查询信息或需要最新数据，只是表达情绪。因此，不需要使用<search>标签。应该直接给予同理心的回应，比如认可他们的心情，并给予积极的回应。不需要任何搜索，直接回答即可。
</think>

听起来很高兴呢！心情好的时候做什么事情都更有动力，祝你今天一切顺利！😊<|im_end|>

✅ Clean_responses (最终答案):
听起来很高兴呢！心情好的时候做什么事情都更有动力，祝你今天一切顺利！😊


In [5]:
print("\n### Test 2: 特斯拉股价 ###")
user_query = "查询特斯拉的实时股价"
rounds2, sources2 = run_search_agent(user_query, max_rounds=5)
show_rounds(rounds2, sources2, user_query)



### Test 2: 特斯拉股价 ###
Query: 查询特斯拉的实时股价

=== Round 1 ===
Raw_responses:
<think>
好的，用户问的是查询特斯拉的实时股价。首先，我需要确定是否需要搜索。根据核心判断原则，实时股价属于需要实时数据的情况，所以必须搜索。接下来，检查是否需要使用<search>标签。用户明确要求查询，所以符合第五条情况，必须使用搜索。然后，确定查询词，应该简短，比如“特斯拉 实时股价”或者“特斯拉股票价格”。确保查询词在3-10字之间，所以可能需要调整，比如“特斯拉股价”更简洁。最后，确保没有编造信息，直接输出搜索标签。不需要其他内容，因为用户只需要搜索结果。
</think>

<search>特斯拉股价</search>

🔍 search_content: 特斯拉股价

📚 搜索到 5 条信息（仅展示前3条）:
  信息1: TSLA的最新盘后股价为454.61，较前一交易日下跌-0.39，跌幅为-0.09%。 查找股票交易商. 实时行情 ......
  信息2: 35,821.04% ... 圖表並未反映夜盤價格。 前收市價 454.48; 開市 452.97; 買入價 448.88 x 200; 賣出價 449.59 x 200; 今日波幅 451.66 ....
  信息3: 新浪财经为您提供特斯拉汽车(TSLA)实时行情数据。包括最新股价、今日涨跌幅、技术走势图、历史数据查询及最新相关新闻资讯，助力您把握美股市场投资机会。...


=== Round 2 ===
Raw_responses:
<think>
好的，我现在需要处理用户的问题：“查询特斯拉的实时股价”。首先，根据之前设定的核心判断原则，用户明确要求查询实时数据，所以必须使用<search>标签搜索。不过，在之前的任务回顾中，用户已经进行了搜索，并获得了五个搜索结果。现在需要分析这些结果是否足够回答问题。

查看搜索结果，信息1提到特斯拉(TSLA)的最新盘后股价为454.61美元，较前一交易日下跌-0.39美元，跌幅-0.09%。信息2显示前收市价为454.48美元，今日波幅451.66，但未提及当前股价。信息3和4是东方财富和新浪财经的链接，但没有具体数字。信息5提到特斯拉454.53美元，涨幅1.74%，但看起来可能是旧数据或

In [9]:
print("\n### Test 3: 奥运会奖牌榜 ###")
user_query = "请告诉我2024年夏季奥运会的最新奖牌榜排名情况"
rounds3, sources3 = run_search_agent(user_query, max_rounds=7)
show_rounds(rounds3, sources3, user_query)


### Test 3: 奥运会奖牌榜 ###


Query: 请告诉我2024年夏季奥运会的最新奖牌榜排名情况

=== Round 1 ===
Raw_responses:
<think>
好的，用户问的是2024年夏季奥运会的最新奖牌榜排名情况。首先，我需要确定是否需要搜索。根据核心判断原则，实时数据和最新信息需要搜索。2024年奥运会如果已经举行，奖牌榜会实时变化，所以需要最新数据。但当前时间是2023年，可能用户的问题是假设性的或者提前想知道。不过根据规则，如果用户询问的是“最新”，而当前还没有举办，可能需要确认。但根据常识，2024年奥运会将在巴黎举行，目前还未开始，所以奖牌榜可能不存在。但用户可能是在测试或者假设情况下提问。根据规则，需要搜索来获取最新信息，所以应该使用<search>标签。查询词应该是“2024夏季奥运会奖牌榜”。
</think>

<search>2024夏季奥运会奖牌榜</search>

🔍 search_content: 2024夏季奥运会奖牌榜

📚 搜索到 5 条信息（仅展示前3条）:
  信息1: 2024年巴黎奥运会 奖牌榜 ; 阿尔巴尼亚. ALB. 阿尔巴尼亚 ; 阿尔及利亚. ALG. 阿尔及利亚 ; 阿根廷. ARG. 阿根廷 ; 亚美尼亚. ARM. 亚美尼亚 ; 澳大利亚. AUS...
  信息2: 本届奥运会美国代表团以40金44银42铜126枚奖牌位居第一；中国代表团以40金27银24铜91枚奖牌位列第二，40枚金牌的成绩超越2012年伦敦奥运会39枚金牌的纪录，创造了新的境外参加 ......
  信息3: 完全奖牌榜. 排名; 国家/地区; 总数. 1, 美国 · 40 · 44 · 42 · 126. 2, 中国 · 40 · 27 · 24 · 91. 3, 日本 · 20 · 12 · 13 · ...


=== Round 2 ===
Raw_responses:
<think>
好的，用户需要2024年夏季奥运会的最新奖牌榜排名情况。首先，我得检查已有的搜索结果是否足够回答这个问题。

查看信息1，来自Olympics.com，列出了多个国家的奖牌情况，但具体数字没有详细说明，可能只是国家列表。信息2来自百度百科，提到美国和中国分别以40金居首和第二，总奖牌数126和91。信息3是央视网的，显示美国、中国、日本、澳大利亚等国的奖牌数

In [10]:
print("\n### Test 4: 上海天气 ###")
user_query = "上海2025年11月25日的天气怎么样?"
rounds4, sources4 = run_search_agent(user_query, max_rounds=5)
show_rounds(rounds4, sources4, user_query)


### Test 4: 上海天气 ###


Query: 上海2025年11月25日的天气怎么样?

=== Round 1 ===
Raw_responses:
<think>
好的，用户问的是上海2025年11月25日的天气怎么样。首先，我需要确定是否需要搜索。根据核心判断原则，实时数据或需要最新信息的情况必须搜索。天气属于实时数据，而且日期是未来的，2025年11月25日还没有到，所以需要查找未来的天气预报。不过，通常天气预报可能不会提前那么久发布，但用户可能希望了解预测情况。因此，必须使用<search>标签来获取信息。查询词应该是“上海2025年11月25日天气”，符合3-10字的要求。不需要编造任何信息，直接搜索即可。
</think>

<search>上海2025年11月25日天气</search>

🔍 search_content: 上海2025年11月25日天气

📚 搜索到 5 条信息（仅展示前3条）:
  信息1: ... 上海天气栏目提供上海天气预报、上海市天气、上海 ... 上海11月25日天气预报(2025). 白天. 小雨. 最高：20℃. 夜间. 阴. 最低：17 ......
  信息2: 上海2025年11月份历史天气 ; 星期二. 2025-11-25 天气. 晴. 5℃ ～ 14℃ ; 星期三. 2025-11-26 天气. 晴转多云. 8℃ ～ 16℃ ; 星期四. 2025-1...
  信息3: 溫度圖. °F. 80° 70° 60° 50° 40° 11月 ... 未來. 十二月2025 · 一月2026 · 二月2026. 全球範圍. 颶風 · 惡劣天氣 · 雷達與氣象圖 · 視訊....


=== Round 2 ===
Raw_responses:
<think>
好的，用户问的是上海2025年11月25日的天气情况。我需要先看看已有的搜索结果是否足够回答这个问题。

首先看信息1，里面提到上海11月25日天气预报，白天小雨，最高20℃，夜间阴，最低17℃。这个信息很直接，给出了具体的温度和天气状况，看起来很准确。不过需要确认是否是2025年的数据，因为信息标题里明确写了2025年，所以应该没问题。

信息2显示2025年11月25日天气是晴，温度5℃到14℃。这里和信息1的天气情况不一致，一个说小雨，一个说晴，温度范围也不同。这可能是因为不同的来

In [11]:
print("\\n### Test 5: Llama3")
user_query = "介绍一下最近很火的AI模型Llama3，它的主要特点、技术架构和最新的应用有哪些？请把它涉及的主要技术点都给我介绍清楚。"
rounds5, sources5 = run_search_agent(user_query, max_rounds=5)
show_rounds(rounds5, sources5, user_query)


\n### Test 5: Llama3
Query: 介绍一下最近很火的AI模型Llama3，它的主要特点、技术架构和最新的应用有哪些？请把它涉及的主要技术点都给我介绍清楚。

=== Round 1 ===
Raw_responses:
<think>
好的，用户问的是关于Llama3这个AI模型的介绍，包括主要特点、技术架构、最新应用以及主要技术点。首先，我需要确定是否需要搜索这些信息。

首先，Llama3是最近比较火的模型，可能涉及实时数据或最新信息。用户要求的是“最近很火”，这可能意味着需要最新的信息，比如发布日期、最新版本或应用案例。此外，用户还要求详细的技术架构和特点，这些可能不是常识，尤其是具体的参数和架构细节，可能需要查阅最新资料。

接下来，检查是否属于需要搜索的情况。根据核心判断原则，如果问题需要最新信息或具体事实，必须搜索。这里用户的问题涉及最新的应用和详细的技术点，所以需要搜索。同时，用户明确要求查询信息，符合第五条原则。

然后，考虑是否已经有足够的信息。由于Llama3是最近发布的，可能没有足够的公开信息，或者需要确认最新数据。因此，必须进行搜索。需要构造简短的查询词，比如“Llama3 技术特点 应用”，确保覆盖用户的问题点。

最后，确保输出格式正确，使用<search>标签，并且内容在3-10字之间。可能需要多个搜索词来全面覆盖问题，但用户要求的是主要技术点，所以可能需要更精确的查询。不过根据规则，每个搜索词单独一行，所以可能需要多次搜索，但用户的问题可能需要综合多个搜索结果，所以先进行一次全面的搜索，再根据结果进一步处理。
</think>

<search>Llama3 技术特点 应用</search>

🔍 search_content: Llama3 技术特点 应用

📚 搜索到 5 条信息（仅展示前3条）:
  信息1: Llama3模型在不同参数量下对应不同应用场景，如小模型适用于移动端，中等模型适用于云端，大模. 型适用于长时间计算。 2、Llama3模型技术指标与行业影响. Llama3模型在多个 ......
  信息2: Llama 3 引入了更复杂的嵌入技术——VocabParallelEmbedding，它优化了模型并行分片中的词汇分布。 两者都采用了“仅解码器”架构，并以“SwiGLU” 作为激活函数，

In [ ]:
print("\\n### Test 6: 美股七姐妹 ###")
user_query = "查询美股七姐妹的实时股价"
rounds5, sources5 = run_search_agent(user_query, max_rounds=10)
show_rounds(rounds5, sources5, user_query)

\n### Test 6: 美股七姐妹 ###
Query: 查询美股七姐妹的实时股价

=== Round 1 ===
Raw_responses:
<think>
好的，用户的问题是“查询美股七姐妹的实时股价”。首先，我需要确定是否需要搜索。根据核心判断原则，实时股价属于需要实时数据的情况，所以必须搜索。接下来，要确认“七姐妹”指的是哪七家公司。通常，美股中的“七姐妹”指的是七大石油公司，如埃克森美孚、壳牌、BP等。但可能需要确认具体是哪些公司，因为不同的上下文可能有不同的指代。不过，用户明确提到的是美股，所以应该是指美国的七大石油公司。因此，搜索关键词应该是“美股七姐妹实时股价”或者类似的短语。根据格式要求，<search>标签需要简短，所以可能需要调整关键词长度。例如，“美股七姐妹股价”或者“七姐妹公司股价”。但可能需要更精确，比如“七姐妹石油公司股价”。不过用户可能指的是具体的七家公司，所以直接搜索“美股七姐妹实时股价”应该可以获取相关信息。需要确保搜索词足够准确，同时符合字数限制。最终决定使用“美股七姐妹实时股价”作为搜索词，但可能需要缩短为“七姐妹实时股价”或者“美股七姐妹股价”。根据规则，查询词需要3-10字，所以可能需要更简短。例如“七姐妹股价”或“美股七姐妹股价”。但可能不够准确，所以可能需要更明确的关键词，如“七姐妹公司股价”。不过用户的问题已经明确是美股七姐妹，所以可能直接搜索“美股七姐妹股价”即可。最终，确定使用“美股七姐妹股价”作为搜索词，符合字数要求，并且准确涵盖用户需求。
</think>

<search>美股七姐妹股价</search>

🔍 search_content: 美股七姐妹股价

📚 搜索到 5 条信息（仅展示前3条）:
  信息1: 美股的“七姐妹”（Magnificent Seven，又称“科技七雄”）是指当前美股市场中市值最高、影响力最大的七家科技巨头，它们分别是： 苹果（Apple, AAPL）...
  信息2: 美银分析显示，“美股七姐妹”估值泡沫尚未触顶。该组合自2023年3月以来累计涨幅达223%，低于历史泡沫244%的平均涨幅。目前其追踪市盈率为39 ......
  信息3: 美股七姐妹是哪七个 · 1. 苹果（Apple, AAPL） · 2. 微软（Microsoft, MSFT） · 3. 亚